# Импортируем библиотеки и загружаем данные

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')

#Для визуализации
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#вывод максимального количества строк и столбцов
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import scipy.sparse as sparse

#Для моделирования
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

Пробовала 2мя способами удалить дубликаты в обучающей выборке, в итоге это только ухудшило результат на kaggle приблизительно до 0.73. В итоге ничего не удаляла

In [ ]:
#train = train.drop_duplicates().reset_index(drop = True)
#train.drop_duplicates(subset=['userid', 'itemid'], keep='last', inplace=True)

# Строим разреженную матрицу взаимодействия

Ниже модель из бэйзлайна, с подобранными руками гиперпараметрами, результат на kaggle 0.75125

In [3]:
#разбиваем на обучающую и тестовую выборки
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)

In [4]:
#делаем спарс матрицу - то есть, разреженную матрицу, нули в которой не хранятся
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

# создание модели

In [5]:
#выводим часть гиперпараметров отдельно
NUM_THREADS = 4 #число потоков
NUM_COMPONENTS = 55 #число параметров вектора 
NUM_EPOCHS = 15 #число эпох обучения

In [6]:
#определяем модель
model_t = LightFM(learning_rate=0.115, loss='logistic',
                no_components=NUM_COMPONENTS, random_state=32)

In [7]:
#обучаем модель
model_t = model_t.fit(ratings_coo, epochs=NUM_EPOCHS, num_threads=NUM_THREADS)

In [8]:
#предсказываем рейтинг на тестовой выборке
preds = model_t.predict(test_data.userid.values,
                      test_data.itemid.values)

In [9]:
#считаем roc_auc
sklearn.metrics.roc_auc_score(test_data.rating,preds)

0.7517362831861725

In [10]:
#считаем рейтинг
preds = model_t.predict(test.userid.values,
                      test.itemid.values)

In [11]:
#оцениваем минимальное и максимальное значение
preds.min(), preds.max()

(-24.29817008972168, 24.720426559448242)

In [12]:
#нормируем значения рейтинга
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()

In [13]:
#проверяем, что получилось
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [ ]:
#выгружаем результат для сабмишна
submission['rating']= normalized_preds
submission.to_csv('submission_log.csv', index=False)

Для того, чтобы использовать метрики precision и recall из библиотеки lightfm, надо сначала сделать спарс матрицу (разреженную) для всего датасета train

In [10]:
ratings_coo = sparse.coo_matrix((train['rating'].astype(int),
                                 (train['userid'],
                                  train['itemid']))) 

In [4]:
#с помощью random_train_test_split из LightFM сделать разбивку на тест и трейн
train_coo, test_coo = random_train_test_split(ratings_coo)

In [9]:
#выводим часть гиперпараметров отдельно
NUM_THREADS = 4 #число потоков
NUM_COMPONENTS = 55 #число параметров вектора 
NUM_EPOCHS = 15 #число эпох обучения

In [14]:
#определяем модель
model = LightFM(learning_rate=0.115, loss='logistic', no_components=NUM_COMPONENTS,random_state=32)

In [15]:
#обучаем модель
model = model.fit(train_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [19]:
#  Сделаем предсказание на тестовой выборке
preds = model.predict(test_coo.row,
                      test_coo.col)


In [20]:
# Посмотрим на метрику roc_auc_score
sklearn.metrics.roc_auc_score(test_coo.data,preds)

0.7564548187843096

In [21]:
# Посмотрим на метрику из LighfFM:
auc_LFM = auc_score(model=model, test_interactions=test_coo)
auc_LFM.max(), auc_LFM.min(), auc_LFM.mean()

(1.0, 0.0, 0.7022203)

метрики roc_auc из sklearn и lightfm существенно отличаются, посмотрим какой результат будет на kaggle

In [22]:
# Посчитаем precision, recall.
pr_at_k = precision_at_k(model=model, test_interactions=test_coo, k=5)
rc_at_k = recall_at_k(model=model, test_interactions=test_coo, k=5)
print(f'Precision: {pr_at_k.mean()}, recall: {rc_at_k.mean()}')

Precision: 0.020512936636805534, recall: 0.061677793206188894


In [23]:
#считаем рейтинг
preds = model.predict(test.userid.values,
                      test.itemid.values)

In [24]:
#оцениваем минимальное и максимальное значение
preds.min(), preds.max()

(-24.599472045898438, 30.27789878845215)

In [25]:
#нормируем значения рейтинга
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()

In [26]:
#проверяем, что получилось
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [27]:
#выгружаем результат для сабмишна
submission['rating']= normalized_preds
submission.to_csv('submission_log.csv', index=False)

результат на kaggle оказался гораздо лучше, чем при первом подходе, гиперпараметры остались неизменными 0.75604

## Рекомендации для пользователя

Для существующих пользователей дадим рекомендации на основании их покупок, а новым пользователям предложим наиболее популярные товары

получим эмбеддинги и запишем их в файл

In [28]:
#Сохранение векторных представлений айтемов для дальнейшего деплоя в продакшн
#item_biases, item_embeddings = model.get_item_representations(features=item_features)
item_biases, item_embeddings = model.get_item_representations()
user_biases, user_embeddings = model.get_user_representations()
user_embeddings.max(), item_embeddings.max()

(1.6982001, 3.1139927)

In [29]:
pip install nmslib

     |████████████████████████████████| 13.0 MB 2.0 MB/s eta 0:00:01    |██▎                             | 942 kB 883 kB/s eta 0:00:14                         | 1.6 MB 883 kB/s eta 0:00:13     |██████▋                         | 2.7 MB 883 kB/s eta 0:00:12
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [30]:
import nmslib
 
#Создаём граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Добавляем товары в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [31]:
#Вспомогательная функция для поиска по графу
def nearest_item_nms(item_id, index, n=5):
    nn = index.knnQuery(item_embeddings[item_id], k=n)
    return nn

In [32]:
# сохраним эмбеддинги для прототипа
import pickle
with open('item_embeddings.pickle', 'wb') as file:
    pickle.dump(item_embeddings, file, protocol=pickle.HIGHEST_PROTOCOL)

# подбор гиперпараметров

## skopt

очень долго считалось, ушло все время GPU, в бэкграунде - save and run all запускала, так же не посчиталось

In [ ]:
from skopt import forest_minimize

In [ ]:
def objective(params):
    try:
        
        print(params) #добавила вывод параметров на печать
    # unpack
        epochs, learning_rate,\
        no_components, alpha = params
    
        user_alpha = alpha
        item_alpha = alpha
        model = LightFM(loss='logistic',#пробовала warp
                        random_state=32,
                        learning_rate=learning_rate,
                        no_components=no_components,
                        user_alpha=user_alpha,
                        item_alpha=item_alpha)
        model.fit(train_coo, epochs=epochs,#вот тут train_coo 
                  num_threads=4,verbose=True) 
    #есть смысл переделать для roc_auc так как в задании он
        patks = precision_at_k(model, test_coo,#вот тут подставила test_coo
                                                  train_interactions=None,
                                                  k=5, num_threads=4)
        mapatk = np.mean(patks)
    # меняем знак на минус, так как нам надо минимизировать
        out = -mapatk
    # откопала в статье,н=здесь ни на что не влияет, но пусть будет
        if np.abs(out + 1) < 0.01 or out < -1.0:
            return 0.0
        else:
            return out
        
    except ValueError as err: #все время лезла ошибка с таким типом из-за слишком маленьких значений, добавила try except, чтобы считалось
        print(err)
        return 9999.0

In [ ]:
space = [(1, 260), # epochs 
         (10**-4, 10**-1, 'log-uniform'), # learning_rate 
         (20, 200), # no_components 
         (10**-6, 10**-1, 'log-uniform'), # alpha 
        ]

res_fm = forest_minimize(objective, space, n_calls=250,
                     random_state=0, verbose=True) 

In [ ]:
print('Maximimum p@k found: {:6.5f}'.format(-res_fm.fun))
print('Optimal parameters:')
params = ['epochs', 'learning_rate', 'no_components', 'alpha']
for (p, x_) in zip(params, res_fm.x):
    print('{}: {}'.format(p, x_))

## optuna

c optuna пока не все получилось, [нашла статью](https://www.eigentheories.com/blog/lightfm-vs-hybridsvd/) но метод fit_params устарел, а аналогов не нашла пока, буду разбираться

In [1]:
!pip install optuna

In [2]:
import optuna
try: # import lightweight progressbar
    from ipypb import track
except ImportError: # fallback to default
    from tqdm.auto import tqdm as track

In [4]:
def evaluate_lightfm(model):
    '''Convenience function for evaluating LightFM.
    It disables user bias terms to improve quality in cold start.'''
    model._model.user_biases *= 0.0
    return model.evaluate()

def find_target_metric(metrics, target_metric):
    'Convenience function to quickly extract the required metric.'
    for metric in metrics:
        if hasattr(metric, target_metric):
            return getattr(metric, target_metric)

def lightfm_objective(model, target_metric):
    'Objective function factory for optuna trials.'
    def objective(trial):
        # sample hyper-parameter values
        model.rank = trial.suggest_int('rank', 1, max_rank)
        model.item_alpha = trial.suggest_loguniform('item_alpha', 1e-10, 1e-0)
        # train model silently and evaluate
        model.verbose = False
        model.build()
        metrics = evaluate_lightfm(model)
        target = find_target_metric(metrics, target_metric)
        # store trial-specific information for later use
        trial.set_user_attr('epochs', model.fit_params['epochs'])
        #fit(X, y=None, *, groups=None, **fit_params)
        trial.set_user_attr('metrics', metrics)
        return target
    return objective

In [5]:
n_trials = {
# epochs: # trials
    15: 30,
    25: 25,
    50: 20,
    75: 15,
    100: 10,
    150: 5
}

In [ ]:
target_metric = 'precision'
objective = lightfm_objective(model, target_metric)

study = optuna.create_study(
    direction = 'maximize',
    sampler = optuna.samplers.TPESampler(seed=32)
)

optuna.logging.disable_default_handler() # do not report progress
for num_epochs, num_trials in track(n_trials.items()):
    model.fit_params['epochs'] = num_epochs
    study.optimize(objective, n_trials=num_trials, n_jobs=1, catch=None)

print(f'The best value of {target_metric}={study.best_value:0.4f} was achieved with '
      f'rank={study.best_params["rank"]} and item_alpha={study.best_params["item_alpha"]:.02e} '
      f'within {study.best_trial.user_attrs["epochs"]} epochs.')